<a href="https://colab.research.google.com/github/DanielFernandoMC/Prueba_Tecnica_Totto-/blob/main/Desarrollo_Prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Se realiza el proceso ETL de los datos

In [6]:
import pandas as pd

# Cargar archivos CSV con fechas parseadas
ventas = pd.read_csv("ventas.csv", parse_dates=["fecha_venta"], sep="," )
clientes = pd.read_csv("clientes.csv", parse_dates=["fecha_registro"], sep=",")
tiendas = pd.read_csv("tiendas.csv", parse_dates=["fecha_apertura"], sep=",")
productos = pd.read_csv("productos.csv", sep=",")

# Normalizar nombres de columnas (snake_case)
ventas.columns = ventas.columns.str.lower().str.strip().str.replace(" ", "_")
clientes.columns = clientes.columns.str.lower().str.strip().str.replace(" ", "_")
tiendas.columns = tiendas.columns.str.lower().str.strip().str.replace(" ", "_")
productos.columns = productos.columns.str.lower().str.strip().str.replace(" ", "_")

# Unificar tipos de ID como string
ventas["id_cliente"] = ventas["id_cliente"].astype(str)
ventas["id_producto"] = ventas["id_producto"].astype(str)
ventas["id_tienda"] = ventas["id_tienda"].astype(str)
clientes["id_cliente"] = clientes["id_cliente"].astype(str)
productos["id_producto"] = productos["id_producto"].astype(str)
tiendas["id_tienda"] = tiendas["id_tienda"].astype(str)

# Añadir columnas derivadas a ventas
ventas["año"] = ventas["fecha_venta"].dt.year
ventas["mes"] = ventas["fecha_venta"].dt.month
ventas["margen_unitario"] = ventas["precio_unitario"] - ventas["descuento"]
ventas["valor_total"] = (ventas["precio_unitario"] - ventas["descuento"]) * ventas["cantidad"]

# Vista previa de los DF
ventas.info()
print(ventas.head())
print()
clientes.info()
print(clientes.head())
print()
tiendas.info()
print(tiendas.head())
print()
productos.info()
print(productos.head())


FileNotFoundError: [Errno 2] No such file or directory: 'ventas.csv'

# 2. Análisis de datos

#### a) Identificar los 5 productos más vendidos por tipo de tienda en el último trimestre (octubre - diciembre 2024), incluyendo el margen de ganancia promedio para cada producto.

In [ ]:
# Fusionar ventas con productos y tiendas
ventas_prod = ventas.merge(productos, on="id_producto", how="left")
ventas_full = ventas_prod.merge(tiendas, on="id_tienda", how="left")

# Filtrar último trimestre 2024
ventas_q4 = ventas_full[
    (ventas_full["fecha_venta"] >= "2024-10-01") &
    (ventas_full["fecha_venta"] <= "2024-12-31")
].copy()

# Calcular margen real unitario
ventas_q4["margen_unitario"] = ventas_q4["precio_unitario"] - ventas_q4["costo_unitario"]

# Agrupar por tipo tienda y producto
agg = (
    ventas_q4.groupby(["tipo_tienda", "id_producto", "nombre_producto"])
    .agg(
        cantidad_total=("cantidad", "sum"),
        margen_promedio=("margen_unitario", "mean")
    )
    .reset_index()
)

# Top 5 por tipo de tienda
top5_por_tipo = agg.sort_values(["tipo_tienda", "cantidad_total"], ascending=[True, False])
top5_final = top5_por_tipo.groupby("tipo_tienda").head(5).reset_index(drop=True)
print(top5_final)

#### b) Desarrolle un análisis de cohorte mensual para evaluar la retención de clientes. Defina la cohorte como el mes de primera compra de cliente.


In [ ]:
# Unir ventas con fecha de venta y cliente
ventas_clientes = ventas.merge(clientes, on="id_cliente", how="left")

# Obtener la fecha de primera compra (cohorte)
ventas_clientes["cohorte"] = ventas_clientes.groupby("id_cliente")["fecha_venta"].transform("min")

# Extraer año-mes
ventas_clientes["cohorte_m"] = ventas_clientes["cohorte"].dt.to_period("M")
ventas_clientes["venta_m"] = ventas_clientes["fecha_venta"].dt.to_period("M")

# Calcular el índice de retención mensual
cohort_data = ventas_clientes.groupby(["cohorte_m", "venta_m"])["id_cliente"].nunique().reset_index()

# Pivotear: cohorte como fila, mes de venta como columna
cohort_pivot = cohort_data.pivot(index="cohorte_m", columns="venta_m", values="id_cliente")

# Normalizar por la primera columna (mes de adquisición)
cohort_retention = cohort_pivot.divide(cohort_pivot.iloc[:, 0], axis=0).round(3)

print(cohort_retention)


Interpretación:

Por ejemplo, para la cohorte de enero 2023:

100% compró ese mes.

96.4% compró en marzo 2023.

~95–97% continuó comprando durante todo el periodo hasta diciembre 2024.

Esto indica una retención bastante alta, lo cual es atípico si los datos fueran reales, y podría deberse a que se usaron datos simulados o replicados.

#### c) Identifique ventas atípicas que podrían indicar errores de sistema o fraude. Utilice técnicas estadísticas (IQR, Z-score) o machine learning (Isolation Forest). Considere variables como cantidad, precio y comportamiento histórico.


In [ ]:
from sklearn.ensemble import IsolationForest
import numpy as np

# Selección de variables relevantes para detectar anomalías
variables = ventas[["cantidad", "precio_unitario", "valor_total"]].copy()

# Reemplazar valores nulos si los hubiera (por ejemplo, por la mediana)
variables = variables.fillna(variables.median(numeric_only=True))

# -----------------------
# MÉTODO 1: Z-score
z_scores = (variables - variables.mean()) / variables.std()
outliers_z = (np.abs(z_scores) > 3).any(axis=1)
ventas["outlier_z"] = outliers_z

# -----------------------
# MÉTODO 2: IQR
Q1 = variables.quantile(0.25)
Q3 = variables.quantile(0.75)
IQR = Q3 - Q1
outliers_iqr = ((variables < (Q1 - 1.5 * IQR)) | (variables > (Q3 + 1.5 * IQR))).any(axis=1)
ventas["outlier_iqr"] = outliers_iqr

# -----------------------
# MÉTODO 3: Isolation Forest
modelo_iso = IsolationForest(contamination=0.01, random_state=42)
ventas["outlier_iforest"] = modelo_iso.fit_predict(variables) == -1

# Consolidar resultados
outliers_detectados = ventas[ventas[["outlier_z", "outlier_iqr", "outlier_iforest"]].any(axis=1)]

